In [ ]:
from PIL import Image
from IPython.display import display
import sys
sys.path.append("../build/bindings/python/")
from pyfluxrt import *

In [ ]:
scene = Scene()

In [ ]:
#scene_file = "../scenes/toml/transform/transform_rotate_cube.toml"
#scene_file = "../scenes/toml/cornell/cornell_pieta.toml"
scene_file = "../scenes/toml/sphere.toml"
loadSceneFromFile(scene, scene_file)
#scene.print()

In [ ]:
# TEMP Shrink scene so the notebook doesn't hang a long time
scene.sensor.pixelwidth = scene.sensor.pixelwidth // 4
scene.sensor.pixelheight = scene.sensor.pixelheight // 4
scene.sensor.print() # FIXME - no output

In [ ]:
renderer = Renderer()
renderer.printConfiguration() # FIXME - no output

In [ ]:
spp = 5

In [ ]:
x = 5
y = 5
pixelCenter = vec2(x, y) + vec2(0.5, 0.5)
standardPixel = scene.sensor.pixelStandardImageLocation(pixelCenter)
blurCoord = vec2(0.0, 0.0)
ray = scene.camera.rayThroughStandardImagePlane(standardPixel, blurCoord)


In [ ]:
print(f"ray origin {ray.origin} direction {ray.direction}")

In [ ]:
intersection = RayIntersection()
pixelRadiance = RadianceRGB()
rng = RNG()
hit = renderer.traceCameraRay(scene, rng, ray, 0.0, 1, [ VaccuumMedium ], intersection, pixelRadiance)
hit

In [ ]:
intersection

In [ ]:
image = Image.new("RGB", (scene.sensor.pixelwidth, scene.sensor.pixelheight))

def renderPixel(x, y, thread_index):
    num_samples = 20
    #print(f"renderPixel(x={x}, y={y}, thread_index={thread_index})")
    pixelCenter = vec2(x, y) + vec2(0.5, 0.5)
    standardPixel = scene.sensor.pixelStandardImageLocation(pixelCenter)
    accumRadiance = RadianceRGB()
    for sample_index in range(0, num_samples):
        blurCoord = vec2(0.0, 0.0)
        ray = scene.camera.rayThroughStandardImagePlane(standardPixel, blurCoord)
        intersection = RayIntersection()
        pixelRadiance = RadianceRGB()
        rng = RNG()
        hit = renderer.traceCameraRay(scene, rng, ray, 0.0, 1,
                                    [ VaccuumMedium ], intersection, pixelRadiance)
        if hit:
            accumRadiance += pixelRadiance
    accumRadiance /= num_samples
    rgb = tuple(int((c + 1) * 0.5 * 255)
                for c in (accumRadiance.r, accumRadiance.g, accumRadiance.b))
    image.putpixel((x, y), rgb)

scene.sensor.forEachPixel(renderPixel)
size = (4 * scene.sensor.pixelwidth, 4 * scene.sensor.pixelheight)
zoomed = image.resize(size, Image.BOX)
display(image)
display(zoomed)